# Import Packages

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime

# Scrape NYT article for Stay at Home data

## Initialize Beautiful Soup Object

In [2]:
# Get URL
URL = 'https://www.nytimes.com/interactive/2020/us/coronavirus-stay-at-home-order.html'
page = requests.get(URL)

In [3]:
# Initialize Soup Object
soup = BeautifulSoup(page.content, 'html.parser')

## Parse HTML & Format Results

In [4]:
# Dig into list-wrap element of html
results = soup.find('div', class_= 'list-wrap')

In [5]:
# Each state is divided by this header
state_orders = results.find_all('div', class_='state-wrap statewide')

In [6]:
# Pull the specific elements where the data is that we need & extract the info
State = []
population_elem = []
order_date_elem = []

for state_orders in state_orders:
    State.append(state_orders['data-state'])
    population_elem.append(state_orders.find('span', class_='l-population').text)
    order_date_elem.append(state_orders.find('span', class_='l-date').text)

In [7]:
# Format the population data into a list
Population = []

for population_elem in population_elem:
    text = population_elem.split()
    if text[2] == 'million':
        Population.append(int(float(text[1]) * 1000000))
    else:
        Population.append(int(text[1].replace(',', '')))

In [8]:
# Format the date information into a list
SAH_Month = []
SAH_Day = []

for order_date_elem in order_date_elem:
    text = order_date_elem.split()
    SAH_Month.append(text[2])
    SAH_Day.append(text[3])

for i in range(len(SAH_Month)):
    datetime_object = datetime.datetime.strptime(SAH_Month[i], "%B")
    month_number = datetime_object.month
    SAH_Month[i] = month_number

In [9]:
# Create the Year list to match the others
SAH_Year = []
for i in range(len(SAH_Month)):
    SAH_Year.append(2020)

In [10]:
# Create date column
date = pd.DataFrame({'year' : SAH_Year,
                     'month' : SAH_Month,
                     'day' : SAH_Day})
Date = pd.to_datetime(date[['month', 'day', 'year']])

In [11]:
# Create data frame
dict = {'State' : State, 
        'SAH_Order_Date' : Date,
        'Population' : Population
       }
SAH_data = pd.DataFrame(dict)
SAH_data

,State,SAH_Order_Date,Population
0,AL,2020-04-04,4900000
1,AK,2020-03-28,737000
2,AZ,2020-03-31,7200000
3,CA,2020-03-19,39600000
4,CO,2020-03-26,5700000
5,CT,2020-03-23,3600000
6,DE,2020-03-24,973000
7,DC,2020-04-01,702000
8,FL,2020-04-03,21500000
9,GA,2020-04-03,10600000


# Import COVID data

In [13]:
import os
os.getcwd()

'C:\\Users\\camco\\Python Class\\py4ds'

In [18]:
states = pd.read_csv('C:/Users/camco/Downloads/us_states_covid19_daily.csv')
states.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,hospitalized,total,totalTestResults,posNeg,fips,deathIncrease,hospitalizedIncrease,negativeIncrease,positiveIncrease,totalTestResultsIncrease
0,20200414,AK,285.0,8063.0,NaN,NaN,32.0,NaN,NaN,NaN,...,32.0,8348.0,8348.0,8348.0,2,1.0,0.0,510.0,8.0,518.0
1,20200414,AL,3876.0,29241.0,NaN,NaN,493.0,NaN,210.0,NaN,...,493.0,33117.0,33117.0,33117.0,1,11.0,36.0,3793.0,142.0,3935.0
2,20200414,AR,1480.0,19651.0,NaN,81.0,130.0,NaN,43.0,29.0,...,130.0,21131.0,21131.0,21131.0,5,0.0,0.0,257.0,70.0,327.0
3,20200414,AS,0.0,3.0,17.0,NaN,NaN,NaN,NaN,NaN,...,NaN,20.0,3.0,3.0,60,0.0,0.0,0.0,0.0,0.0
4,20200414,AZ,3806.0,40290.0,NaN,580.0,NaN,286.0,NaN,216.0,...,NaN,44096.0,44096.0,44096.0,4,9.0,0.0,645.0,104.0,749.0
